# Introduction

This notebook presents how to upload time waveform vibration data from vibration sensors to Multiviz using a Python client.
The notebook covers the following topics:
- Setting up the Multiviz client
- Data storage and format
- Creating waveform sources
- Uploading waveform measurements

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
project_root = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath('')), '..'))

if project_root not in sys.path:
    print("Adding project root to sys.path:", project_root)
    sys.path.insert(0, project_root)

In [ ]:
from src.multiviz_client import (
    MultivizClient,
)

# Constants
MULTIVIZ_BASE_URL = "https://api.beta.multiviz.com"
MULTIVIZ_API_KEY = ""  # We will provide the API key separately.

# Configuration Multivize client
client = MultivizClient(
    base_url=MULTIVIZ_BASE_URL,
    api_key=MULTIVIZ_API_KEY,
)
TZ = "Europe/Stockholm"
DATE_FORMAT = "%Y-%m-%d %H:%M:%S"

# Data, storage and format
MultiViz uses a source-measurement model to store and analyze time waveform vibration data.
A sources represents a single data stream (axis) from a sensor (machine spot), while a measurement represents a single recording of data from that source.
This means that all the data for a single axial sensor is stored in a single waveform source or simply source.
For example, if you have a single axial wired sensor, you need to first create a source to be able to upload the data from your sensor.

Furthermore, if you have a tri-axial sensor, you need to create three sources, one for X, one for Y and one for Z.

To create a source you need to send a POST request to: https://api.beta.multiviz.com/sources/ 

You need to provide an id known as external_id.
This is the id that you can use to be able to refer to this source in the future.
Typically, you can use a combination of sensor SN (serial number) and axis (or channel) as the external_id.

As result, you will receive the source_id. You will need it to create a measurements or updating the source meta.

Optional:
All sources and sensors can be organized by building an Asset Hierarchy or Tree.
For Multiviz to be able to create an asset tree on the dashboard, you can also include optional meta data:

```json
"meta": {
        "location": "building A",
        "assetName": "Motor A",
        "sensorName": "NDA",
        "measurementName": "X",
        "unitOfMeasure": "g",
        "sourceProfile": "default",
}
```

**Notes:**
- `location`: The name of location such as building or area name.
- `sensorName`: Usually the name of the measuring point, such as DE or NDE but it is up to the user.
- `measurementName`: You can use the axis (X, Y, Z), the orientation (Axial, Vertical, Horizontal) or channel number (for wired systems).
- `unitOfMeasure`: "g", " could be "g", or "mss".
- `sourceProfile`: For the wireless or wired sensors, you should use "default". For data from handheld analyzers please use "handheld".

In [ ]:
def create_waveform_source(
    external_id: str, sensor_id: str, axis: str, source_metadata: dict
):
    """
    Create a waveform source in MultiViz.

    """
    payload = {
        "meta": {
            "location": source_metadata.get("location", "Unknown Location"),
            "assetName": source_metadata.get("assetName", "Unknown Asset"),
            "sensorName": source_metadata.get("sensorName", "Unknown Sensor"),
            "measurementName": axis,
            "unitOfMeasure": source_metadata.get("unitOfMeasure", "g"),
            "sourceProfile": source_metadata.get("sourceProfile", "default"),
        },
        "external_id": external_id,
        "sensor_id": sensor_id,
        "sensor_stream_id": axis,
        "channels": ["acc"],
    }
    return client.create_waveform_source(
        payload=payload,
        ignore_existing=True,
    )

If for any reason you would like to update the source metadata later, you can use this function:

In [ ]:
def update_waveform_source_meta(source_id: str, source_metadata: dict):
    """
    Update the waveform source metadata.
    Updating other fields like sensor_id, sensor_stream_id, channels is not supported.
    """
    payload = {
        "meta": {
            "location": source_metadata.get("location", "Unknown Location"),
            "assetName": source_metadata.get("assetName", "Unknown Asset"),
            "sensorName": source_metadata.get("sensorName", "Unknown Sensor"),
            "measurementName": source_metadata.get("measurementName", "acc"),
            "unitOfMeasure": source_metadata.get("unitOfMeasure", "g"),
            "sourceProfile": "default",
        },
    }
    return client.update_source(
        source_id=source_id,
        payload=payload,
    )

## Waveform Measurement
To upload the measurement, one needs to specify the source_id, representing a bucket of data, to upload the measurement to.

Additionally, the timestamp of the measurement and duration are mandatory.

You need to send a POST request to: https://api.beta.multiviz.com/sources/{source_id}/measurements


The timestamp should be in milliseconds and in UTC format and integer value.
The duration should be in seconds and float value.
If the duration is not known, it can be calculated by dividing the number of samples by the sampling rate.

It's also required to provide the data as a list of acceleration values in the unit of the source.

If there are any scalar values (e.g., temperature, battery, RPM, etc.) associated with the measurement, they can be included in an optional dictionary.

```json
payload={
    "source_id": source_id,
    "timestamp": timestamp, # should be in milliseconds and in UTC format
    "duration": duration, # in seconds
    "data": data, # list of acceleration values in the unit of source
    "scalars": scalars, # optional scalar values dictionary (e.g., temperature, battery, RPM, etc.
}
```

In [ ]:
def upload_waveform_measurement(
    source_id: str,
    timestamp: int,
    duration: float,
    data: list,
    meta: dict = None,
    scalars: dict = None,
):
    """
    Upload acceleration waveform measurement.

    """

    sample_rate = int(len(data) / duration)
    payload = {
        "timestamp": timestamp,
        "duration": duration,
        "data": data,
        "meta": {"sample_rate": sample_rate, **(meta or {})},
        "scalars": {**(scalars or {})},
    }
    print(payload)
    return client.create_waveform_measurement(
        source_id=source_id,
        payload=[payload],
        ignore_existing=True,
    )

## Example 1: SJON
Here is an example of uploading a measurement.
On this first example, the data is originally available on the JSON format.


In [ ]:
from src.helper import load_json_payload, localize_timestamp, str_clean


def upload_example_measurement_json():
    # Load sample payload
    # It could be anything like CSV, JSON, etc. Here we are using JSON for simplicity.
    sample_payload = load_json_payload("../data/sample_payload.json")

    # Extract source and measurement info
    sensor_id = sample_payload["sensor_id"]

    existing_axis = ["X", "Y", "Z"]
    for axis in existing_axis:
        external_id = f"{str_clean(sensor_id)}_{str_clean(axis)}"  # e.g., sensor123_x
        source_metadata = {
            "location": sample_payload.get("location", "Unknown Location"),
            "assetName": sample_payload.get("machine", "Unknown Machine"),
            "sensorName": sample_payload.get("sensor_name", "Unknown Sensor"),
            "measurementName": axis,
            "unitOfMeasure": sample_payload.get("unit", "g"),
            "sourceProfile": "default",
        }

        print(
            f"Processing external_id: {external_id} for axis: {axis}, source_metadata: {source_metadata}"
        )

        # creating a source
        result = create_waveform_source(
            external_id=external_id,
            sensor_id=sensor_id,
            axis=axis,
            source_metadata=source_metadata,
        )

        print(f"Using source_id: {result.get('source_id')} for axis: {axis}")

        measureent_date = sample_payload["date"]
        timestamp = localize_timestamp(measureent_date, TZ, DATE_FORMAT)
        duration = sample_payload["duration"]
        data = sample_payload[axis]

        data = {"acc": data}

        # prepare meta and scalar values
        # you can also pass empty dicts if you don't have any meta or scalar values
        meta = {
            "gateway_id": sample_payload.get("gateway_id", "Unknown Gateway"),
        }
        scalars = {
            "temperature": sample_payload.get("process_data", {}).get("temperature"),
            "battery": sample_payload.get("process_data", {}).get("battery"),
        }  # Add other scalar values as needed

        # create measurement
        upload_waveform_measurement(
            result.get("source_id"), timestamp, duration, data, meta=meta, scalars=scalars
        )

In [ ]:
upload_example_measurement_json()

## Example 2: CSV
Here is a second example for uploading a measurement.
On this example, we have provided example csv file.
In addition, we provide a txt file for source meta information.

**Notes:**
- `values_1.csv`, `values_2.csv`, `values_3.csv`: These files contain timewave data for X, Y, and Z axes, respectively.
- information.txt: This file contains metadata information such as location, machine name, sensor name, date_time, and sensor_id.

In [ ]:
from datetime import datetime
from pathlib import Path

import pytz
from src.helper import axis_for, parse_information_file, read_timewave_column


def upload_example_measurement_csv():
    # Load information from txt file
    info = parse_information_file(Path("../data/information.txt"))
    print(f"Loaded information: {info}")

    # Extract source and measurement info
    sensor_id = str(info.get("sensor_id", "")).strip()

    for fname in ("../data/values_1.csv", "../data/values_2.csv", "../data/values_3.csv"):
        data = {"acc": read_timewave_column(Path(fname))}
        axis = axis_for(sensor_id, fname)
        sensor_stream_id = axis

        # in this case, we use location from info as part of external_id
        external_id = (
            f"{sensor_id}_{sensor_stream_id}"
        )

        source_metadata = {
            "location": info.get("location", "Unknown Location"),
            "assetName": info.get("machine", "Unknown Machine"),
            "sensorName": info.get("sensor_name", "Unknown Sensor"),
            "measurementName": sensor_stream_id,
            "unitOfMeasure": "g",
            "sourceProfile": "default",
        }

        print(
            f"Processing external_id: {external_id} for axis: {axis}, source_metadata: {source_metadata}"
        )

        # creating a source
        result = create_waveform_source(
            external_id=external_id,
            sensor_id=sensor_id,
            axis=axis,
            source_metadata=source_metadata,
        )

        formatted_date = info.get("date_time", "")
        TZ = pytz.timezone("Europe/Stockholm")
        dt_object = datetime.strptime(formatted_date, "%m/%d/%Y %H:%M:%S")
        local_dt = TZ.localize(dt_object)  # Convert to UTC timezone
        utc_dt = local_dt.astimezone(pytz.UTC)
        timestamp_utc = int(
            utc_dt.timestamp() * 1000
        )  # Convert to Unix timestamp (integer)
        duration = info.get("duration_s", 0)
        frequency_hz = int(info.get("samples", 0) / duration) if duration > 0 else 0

        # prepare meta and scalar values
        # you can also pass empty dicts if you don't have any meta or scalar values

        meta = {"sampling_date": formatted_date, "sampling_rate": frequency_hz}

        scalars = {}  # there is no scalar data in this example

        # create measurement
        upload_waveform_measurement(
            result.get("source_id"), timestamp_utc, duration, data, meta=meta, scalars=scalars
        )

In [ ]:
upload_example_measurement_csv()